<a href="https://colab.research.google.com/github/wespa4/semsPy/blob/main/digit_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Генерация цифр через GAN

In [1]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import torch
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization, Reshape, Conv2D, Conv2DTranspose, ReLU, LeakyReLU,InputLayer, UpSampling2D,ZeroPadding2D
import tensorflow.keras.datasets as dts
from PIL import Image, ImageOps
from tensorflow.keras.utils import to_categorical
from google.colab.patches import cv2_imshow

In [3]:
y_train = np.concatenate((y_train,y_test))
x_train = np.concatenate((x_train,x_test))

In [8]:
x = (x_train.astype(np.float32) - 127.5) / 127.5
x = np.expand_dims(x,-1)

In [9]:
x.shape

(70000, 28, 28, 1)

In [10]:
y = to_categorical(y_train)

In [12]:
y.shape

(70000, 10)

In [13]:
lr = 0.0001
batch_size = 128
betas = (0.5, 0.999)
noise_size = 32

In [14]:
discriminator = keras.Sequential([
  InputLayer((28,28,1)),
  Conv2D(32, kernel_size=3, strides=2, padding="same"),
  LeakyReLU(alpha=0.2),
  Dropout(0.25),
  Conv2D(64, kernel_size=3, strides=2, padding="same"),
  ZeroPadding2D(padding=((0,1),(0,1))),
  BatchNormalization(momentum=0.8,),
  LeakyReLU(alpha=0.2),
  Dropout(0.25),
  Conv2D(256, kernel_size=3, strides=1, padding="same"),
  BatchNormalization(momentum=0.8),
  LeakyReLU(alpha=0.2),
  Dropout(0.25),
  Flatten(),
  Dense(10, activation='sigmoid'),
])

discriminator.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 14, 14, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d (ZeroPadding2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 256)      │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │       163,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 331,658 (1.27 MB)

 Trainable params: 331,018 (1.26 MB)

 Non-trainable params: 640 (2.50 KB)

In [15]:
optimizer = keras.optimizers.Adam(lr,betas[0],betas[1])
discriminator.compile(loss='binary_crossentropy',
  optimizer=optimizer)

In [16]:
generator = keras.Sequential([
  InputLayer((noise_size+10,)),
  Dense(42 * 7 * 7, activation="relu"),
  Reshape((7, 7, 42)),
  UpSampling2D(),
  Conv2D(42, kernel_size=3, padding="same"),
  BatchNormalization(momentum=0.8),
  ReLU(),
  UpSampling2D(),
  Conv2D(42, kernel_size=3, padding="same"),
  BatchNormalization(momentum=0.8),
  ReLU(),
  Conv2D(42, kernel_size=3, padding="same"),
  BatchNormalization(momentum=0.8),
  ReLU(),
  Conv2D(1, kernel_size=3, padding="same", activation='tanh')
])

generator.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 2058)           │        88,494 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 7, 42)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 14, 14, 42)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 42)     │        15,918 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 42)     │           168 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 14, 14, 42)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 28, 28, 42)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 42)     │        15,918 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 42)     │           168 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 28, 28, 42)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 42)     │        15,918 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 28, 28, 42)     │           168 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 28, 28, 42)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 1)      │           379 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 137,131 (535.67 KB)

 Trainable params: 136,879 (534.68 KB)

 Non-trainable params: 252 (1008.00 B)

In [17]:
optimizer = keras.optimizers.Adam(lr,betas[0],betas[1])
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

In [18]:
optimizer = keras.optimizers.Adam(lr,betas[0],betas[1])
z = Input(shape=(noise_size+10,))
img = generator(z)
valid = discriminator(img)
combined = keras.Model(z, valid)

In [19]:
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [22]:
num_epochs = 100

In [23]:
for epoch in range(num_epochs):
  idx = np.random.randint(0,x_train.shape[0], batch_size)
  imgs = x[idx]
  setting = y[idx]
  noise = np.random.normal(0, 1, (batch_size, noise_size))
  settingNoise = np.concatenate((noise,setting),1)
  gen_imgs = generator.predict(settingNoise)

  d_loss_real = discriminator.train_on_batch(imgs, setting)
  d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 10)))
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  setting = to_categorical(np.random.randint(10,size=batch_size))
  noise = np.random.normal(0, 1, (batch_size, noise_size))
  settingNoise = np.concatenate((noise,setting),1)
  valid_y = setting

  discriminator.trainable = False
  g_loss = combined.train_on_batch(settingNoise, valid_y)
  discriminator.trainable = True

  print (epoch, d_loss, g_loss)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step
0 0.07164074 0.22389011
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1 0.0716223 0.22380775
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
2 0.071592376 0.22375187
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
3 0.07158296 0.22367431
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
4 0.07155515 0.22360054
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
5 0.07153695 0.2235514
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
6 0.07152748 0.2234934
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
7 0.07149824 0.22345053
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
8 0.07146923 0.2234066
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
9 0.071457714 0.22335054
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
10 0.07143575 0.22330727
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
11 0.07141504 0.2232481
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
12 0.071396895 0.22320007
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
13 0.07136844 0.22315063
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
14 0.07135485 0.22307938
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
15 0.07132272 0.22304162
4

In [24]:
def genNum(num):
  noise = np.random.normal(0, 1, (1, noise_size))
  setting = to_categorical([num],10)
  settingNoise = np.concatenate((noise,setting),1)
  generated_samples = generator(settingNoise)
  generated_samples = np.array(generated_samples)
  generated_samples = ((generated_samples * 0.5) + 0.5) * 255
  generated_samples = generated_samples.round()
  generated_samples = generated_samples.astype(np.uint8)
  return generated_samples

In [27]:
sample = genNum(3)
cv2_imshow(sample[0])

In [ ]:
discriminator.save('NUMdis_2000.h5')
generator.save('NUMgen_2000.h5')